In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("education vertical dataset.csv")
df

,id,vertical,FS,description,price
0,0,education,IT DEV Specialist,Skilled in programming languages such as Java,54
1,1,education,IT DEV Specialist,Experienced in developing and maintaining com...,55
2,2,education,IT DEV Specialist,Knowledgeable in Agile development methodolog...,53
3,3,education,IT DEV Specialist,Strong problem-solving skills and ability to ...,55
4,4,education,IT DEV Specialist,Proficient in using development tools such as...,52
...,...,...,...,...,...
135,138,education,Pure Math teacher,Strong understanding of the importance of fos...,21
136,139,education,Pure Math teacher,Proficient in using formative and summative a...,23
137,140,education,Pure Math teacher,Experienced in creating engaging and interact...,24
138,141,education,Pure Math teacher,Skilled in providing opportunities for hands-...,23


In [3]:
df = df[['description','price']]
df

,description,price
0,Skilled in programming languages such as Java,54
1,Experienced in developing and maintaining com...,55
2,Knowledgeable in Agile development methodolog...,53
3,Strong problem-solving skills and ability to ...,55
4,Proficient in using development tools such as...,52
...,...,...
135,Strong understanding of the importance of fos...,21
136,Proficient in using formative and summative a...,23
137,Experienced in creating engaging and interact...,24
138,Skilled in providing opportunities for hands-...,23


In [7]:
import re
from numpy import array
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from numpy import array
from numpy import asarray
from numpy import zeros
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [8]:
def get_features(text_series):
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, padding='pre', truncating='pre',maxlen=maxlen)

In [9]:
maxlen = len(df.description.max())
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.description)
max_words = len(tokenizer.word_index) + 1
vocab_size = max_words
x = get_features(df.description)
print(x.shape)
y=df['price']

(140, 73)


In [11]:
# GloVe embedding
dimensions = 300
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B/glove.6B.300d.txt', mode='rt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, dimensions))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [29]:
filter_length = 64
model = Sequential()

e = Embedding(vocab_size, dimensions, weights=[embedding_matrix], input_length=maxlen, trainable=False)
model.add(e)
model.add(Conv1D(filter_length, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Flatten())

model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

model.summary()

callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=10)
]

history = model.fit(x, y,
                    epochs=15,
                    batch_size=1,
                    validation_split=0.01,
                    callbacks=callbacks)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 73, 300)           110100    
                                                                 
 conv1d_3 (Conv1D)           (None, 71, 64)            57664     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 flatten_3 (Flatten)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 167,829
Trainable params: 57,729
Non-trainable params: 110,100
___________________________________________

In [31]:
# use the model to predict a price for a new description
new_description = ["great programmer"]
new_description_sequence = tokenizer.texts_to_sequences([new_description])
new_description_sequence = pad_sequences(new_description_sequence, maxlen=maxlen)
predicted_price = model.predict(new_description_sequence)
print(predicted_price)

1/1 [==============================] - 0s 25ms/step
[[2.0017047]]
